In [ ]:
import pandas as pd
import numpy as np


def read_excel(path="./data.xlsx"):
    data_xls = pd.io.excel.ExcelFile(path)
    data = {}
    print(data_xls.sheet_names)
    for name in data_xls.sheet_names:
        df = pd.read_excel(data_xls, sheet_name=name)
        data[name] = df
        # print(name, " : df = ",df , type(df))
        if df.empty:
            continue
    return data


data = read_excel()
data
pinformation = data["驾驶员基本信息"]
pwork = data["工作时长"]
service_violation = data["服务违章"]
safety_violation = data["安全违章"]
safety_accident = data["安全事故"]
# pinformation
# 男 - 1 ； 女 - 0
pinformation["性别"].value_counts()
pinformation.loc[pinformation["性别"] == '男', '性别'] = 1
pinformation.loc[pinformation["性别"] == '女', '性别'] = 0
pinformation
pinformation["分公司"].value_counts()
pinformation = pinformation.drop(labels="分公司", axis=1)
pinformation
pinformation["学历"].value_counts()
# 普高 0，中专 1， 初中 2， 大专 3， 高中（普高） 4， 本科 5， 技校 6， 职高 7 ，小学 8， nan 9
pinformation.loc[pinformation["学历"] == '普高', '学历'] = 0
pinformation.loc[pinformation["学历"] == '中专', '学历'] = 1
pinformation.loc[pinformation["学历"] == '初中', '学历'] = 2
pinformation.loc[pinformation["学历"] == '大专', '学历'] = 3
pinformation.loc[pinformation["学历"] == '高中（普高）', '学历'] = 4
pinformation.loc[pinformation["学历"] == '本科', '学历'] = 5
pinformation.loc[pinformation["学历"] == '技校', '学历'] = 6
pinformation.loc[pinformation["学历"] == '职高', '学历'] = 7
pinformation.loc[pinformation["学历"] == '小学', '学历'] = 8
pinformation["学历"] = pinformation["学历"].fillna(9)
pinformation
pinformation['出生日期'] = pd.to_datetime(pinformation['出生日期'])
import datetime as dt

now_year = dt.datetime.today().year
# pinformation['age'] = now_year - frame.birth.dt.year
pinformation['出生日期'] = now_year - pinformation['出生日期'].dt.year

pinformation['入职日期'] = pd.to_datetime(pinformation['入职日期'])
now_year = dt.datetime.today().year
# pinformation['age'] = now_year - frame.birth.dt.year
pinformation['入职日期'] = now_year - pinformation['入职日期'].dt.year

pinformation
# pinformation = pinformation.drop(labels=["事故", "安全违章", "服务违章"], axis=1)

safety_accident.columns = safety_accident.columns.str.replace('事故日期', '开班日期')

safety_accident['开班日期'] = pd.to_datetime(safety_accident['开班日期'], format='%Y-%M-%d').dt.date.astype(object)
safety_accident
pwork['开班日期'] = pd.to_datetime(pwork['开班日期'], format='%Y-%M-%d').dt.date.astype(object)
res = pd.merge(pwork, safety_accident, how='right', on=['员工编号', '开班日期'])
res
# print(safety_accident)
import matplotlib.pyplot as plt

y_1 = res['工作时长（小时）']
y_2 = res['驾车时长（小时）']
y_3 = res['运营时长（小时）']
x = [i for i in range(len(y_1))]
plt.plot(x, y_1)
plt.plot(x, y_2)
plt.plot(x, y_3)
plt.show()

print(res)
import seaborn as sns

hres = res[["工作时长（小时）", "驾车时长（小时）", "运营时长（小时）"]]
plot = sns.heatmap(res)
plt.show()
hh = pd.merge(pwork, safety_accident, how='left', on=['员工编号'])
hh.insert(loc=len(hh.columns.array), column="事故", value=1)
hh.loc[hh['事故原因'].isna(), "事故"] = 0
print(hh)
import seaborn as sns

hres = hh[["工作时长（小时）", "驾车时长（小时）", "运营时长（小时）", "事故"]]

plt.subplots(figsize=(20, 15))
ax = plt.axes()
ax.set_title("Correlation Heatmap")
corr = hres.corr()
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()

oth_res = pd.merge(pwork, safety_accident, how='left', on=['员工编号'])
print(oth_res)
oth_y_1 = oth_res[oth_res['开班日期_y'].isna()]['工作时长（小时）']
oth_y_2 = oth_res[oth_res['开班日期_y'].isna()]['驾车时长（小时）']
oth_y_3 = oth_res[oth_res['开班日期_y'].isna()]['运营时长（小时）']
oth_x = [i for i in range(len(oth_y_1))]
plt.subplot(3, 2, 1)
plt.scatter(x, y_1)
plt.subplot(3, 2, 2)
plt.scatter(oth_x, oth_y_1)
plt.show()
plt.subplot(3, 2, 3)
plt.scatter(x, y_2)
plt.subplot(3, 2, 4)
plt.scatter(oth_x, oth_y_2)
plt.show()
plt.subplot(3, 2, 5)
plt.scatter(x, y_3)
plt.subplot(3, 2, 6)
plt.scatter(oth_x, oth_y_3)
plt.show()
print(service_violation.groupby(service_violation['服务违章项目']).value_counts())
service_res = pd.merge(service_violation, safety_accident, how='left', on=['员工编号'])
print(service_res)
no_num = len(service_res[oth_res['开班日期_y'].isna()])
ys_num = len(service_res['服务违章项目']) - no_num
plt.bar([0, 1], [no_num, ys_num], width=0.5)
plt.show()
plt.pie([float(no_num) / float(no_num + ys_num), float(ys_num) / float(no_num + ys_num)])
plt.show()
# pinformation
# pinformation = pinformation.drop(labels=[ "服务违章"], axis=1)
pinformation.insert(loc=6, column="服务违章", value=0)

pinformation.loc[pinformation["员工编号"].isin(service_violation["员工编号"]), "服务违章"] = 1
pinformation
safety_res = pd.merge(safety_violation, safety_accident, how='left', on=['员工编号'])
print(safety_res)
sno_num = len(safety_res[safety_res['开班日期'].isna()])
sys_num = len(safety_res['安全违章项目']) - sno_num
plt.bar([0, 1], [sno_num, sys_num], width=0.5)
plt.show()
plt.pie([float(sno_num) / float(sno_num + sys_num), float(sys_num) / float(sno_num + sys_num)])
plt.show()

pinformation.insert(loc=7, column="安全违章", value=0)
pinformation
pinformation.loc[pinformation["员工编号"].isin(safety_violation["员工编号"]), "安全违章"] = 1
print(pinformation["安全违章"].value_counts())
pinformation.insert(loc=8, column="事故", value=0)
pinformation
pinformation.loc[pinformation["员工编号"].isin(safety_accident["员工编号"]), "事故"] = 1
pinformation
pinformation = pinformation.drop(labels=["初领证日期"], axis=1)
pinformation = pinformation.drop(labels=["员工编号"], axis=1)
# pinformation = pinformation.drop(labels=["入职日期"], axis=1)
pinformation
print(pinformation['事故'].value_counts())
pinformation.to_csv("./data1.csv", index=False)
print(pinformation[pinformation['安全违章'] == 1]['事故'].value_counts())
data_list = pinformation.values.tolist()
print(data_list)
label_list = []
for i in range(len(data_list)):
    label_list.append([data_list[i][-1]])
    data_list[i] = data_list[i][:-1]
print(data_list)
print(label_list)
import torch
import numpy as np
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt

n_data = torch.ones(100,2)
x0 = torch.normal(2*n_data,1)
y0 = torch.zeros(100)
# print(x0)
x1 = torch.normal(-2*n_data,1)
y1 = torch.ones(100)


x = torch.cat((x0,x1),0).type(torch.FloatTensor)
y = torch.cat((y0,y1)).type(torch.LongTensor)
# print(y)
x,y = Variable(x),Variable(y)

# plt.scatter(x[:,0],x[:,1],c=y.data.numpy(), s=100, lw=0, cmap='RdYlGn')
# plt.scatter(x.data.numpy()[:, 0], x.data.numpy()[:, 1], c=y.data.numpy(), s=100, lw=0, cmap='RdYlGn')
# plt.show()

class Net(torch.nn.Module):
    def __init__(self,n_input,n_hidden,n_output):
        super(Net,self).__init__()
        self.hidden1 = torch.nn.Linear(n_input,n_hidden)
        self.hidden2 = torch.nn.Linear(n_hidden,n_hidden)
        self.predict = torch.nn.Linear(n_hidden,n_output)

    def forward(self, input):
        out = self.hidden1(input)
        out = F.sigmoid(out)
        out = self.hidden2(out)
        out = F.sigmoid(out)
        out = self.predict(out)
        # out = F.softmax(out)
        return out

net = Net(6,24,2)
print(net)

optimizer = torch.optim.SGD(net.parameters(),lr=0.02)
loss_func = torch.nn.CrossEntropyLoss()

plt.ion()
plt.show()
y = []
for i in range(len(label_list)):
    y.append(label_list[i][0])
print(torch.tensor(y).shape)
p1 = []
p2 = []
p3 = []
p4 = []
pp = None
for t in range(200):
    prediction = net(torch.tensor(data_list).type(torch.FloatTensor))
    # p11, p22, p33, p44 = num_get(F.softmax(prediction).detach().numpy() , y)
    # p1.append(p11)
    # p2.append(p22)
    # p3.append(p33)
    # p4.append(p44)
    # print(classification_report(prediction.detach().numpy(), y))
    # print(prediction)
    pp = prediction
    print(F.softmax(prediction))
    loss = loss_func(prediction,torch.tensor(y))
    p1.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(t, "  loss: ", loss.item())

    # if t%2==0:
    #     plt.cla()
    #     # 过了一道 softmax 的激励函数后的最大概率才是预测值
    #     # print(F.softmax(prediction))
    #     prediction = torch.max(F.softmax(prediction),1)[1]
    #     pred_y = prediction.data.numpy().squeeze()
    #     target_y = torch.tensor(y).data.numpy()
    #     plt.scatter(torch.tensor(data_list).data.numpy()[:, 0], torch.tensor(data_list).data.numpy()[:, 1], c=pred_y, s=100, lw=0, cmap='RdYlGn')
    #     accuracy = sum(pred_y == target_y) / 200.  # 预测中有多少和真实值一样
    #     plt.text(1.5, -4, 'Accuracy=%.2f' % accuracy, fontdict={'size': 20, 'color': 'red'})
    #     plt.pause(0.1)

plt.ioff()  # 停止画图
plt.show()
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.linear_model import LogisticRegression

# plt.plot([i for i in range(len(p1))], p1)
# plt.show()
res_pre = []
p = []
for i in range(len(y)):
    # print(test[i][-1], y_predict[i])
    pp = pp.detach().numpy()
    for j in range(2):
        if pp[i][j] < 0:
            pp[i][j]=0
    pp = torch.tensor(pp)
    res_pre.append(F.softmax(pp).detach().numpy()[i][int(y[i])])

    print(pp[i], y[i], res_pre[i])
Precision,Recall,throds = precision_recall_curve(y, res_pre)
FPR,TPR,_ = roc_curve(y, res_pre)
# print(y_predict.shape, test[:, -1].shape, Precision,Recall, throds)

plt.xlabel("Recall")
plt.ylabel("Preccision")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.plot(Precision, Recall)
plt.show()
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.plot(FPR, TPR)
plt.show()